## Install Rapids for 500x faster kNN on GPUs

In [1]:
import sys
!cp ../input/rapids/rapids.0.13.0 /opt/conda/envs/rapids.tar.gz
!cd /opt/conda/envs/ && tar -xzvf rapids.tar.gz > /dev/null
sys.path = ["/opt/conda/envs/rapids/lib/python3.6/site-packages"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib/python3.6"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib"] + sys.path
!cp /opt/conda/envs/rapids/lib/libxgboost.so /opt/conda/lib/

In [2]:
import numpy as np
# import pandas as pd
import cudf
import cupy as cp
from cuml.neighbors import KNeighborsRegressor
from cuml import SVR
from cuml.linear_model import Ridge, Lasso
from sklearn.model_selection import KFold
from cuml.linear_model import ElasticNet,LinearRegression

def metric(y_true, y_pred):
    return np.mean(np.sum(np.abs(y_true - y_pred), axis=0)/np.sum(y_true, axis=0))

In [3]:
fnc_df = cudf.read_csv("../input/trends-assessment-prediction/fnc.csv")
loading_df = cudf.read_csv("../input/trends-assessment-prediction/loading.csv")



/opt/conda/envs/rapids/lib/python3.6/site-packages/fsspec/implementations/local.py:33: FutureWarning: The default value of auto_mkdir=True has been deprecated and will be changed to auto_mkdir=False by default in a future release.
  FutureWarning,


In [4]:
loading_df

,Id,IC_01,IC_07,IC_05,IC_16,IC_26,IC_06,IC_10,IC_09,IC_18,...,IC_08,IC_03,IC_21,IC_28,IC_11,IC_20,IC_30,IC_22,IC_29,IC_14
0,10001,0.006070,0.014466,0.004136,0.000658,-0.002742,0.005033,0.016720,0.003484,0.001797,...,0.018246,0.023711,0.009177,-0.013929,0.030696,0.010496,0.002892,-0.023235,0.022177,0.017192
1,10002,0.009087,0.009291,0.007049,-0.002076,-0.002227,0.004605,0.012277,0.002946,0.004086,...,0.014635,0.022556,0.012004,-0.011814,0.022479,0.005739,0.002880,-0.016609,0.025543,0.014524
2,10003,0.008151,0.014684,0.010444,-0.005293,-0.002913,0.015042,0.017745,0.003930,-0.008021,...,0.019565,0.030616,0.018184,-0.010469,0.029799,0.015435,0.005211,-0.028882,0.031427,0.018164
3,10004,0.004675,0.000957,0.006154,-0.000429,-0.001222,0.011755,0.013010,0.000193,0.008075,...,0.002658,0.022266,0.005956,-0.010595,0.024078,-0.000319,0.005866,-0.015182,0.024476,0.014760
4,10005,-0.000398,0.006878,0.009051,0.000369,0.000336,0.010679,0.010352,0.003637,0.004180,...,0.009703,0.017257,0.005454,-0.008591,0.019416,0.000786,0.002692,-0.019814,0.017105,0.013316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11749,21750,0.005996,0.003873,0.012353,0.000242,-0.002159,0.020201,0.020931,0.003684,-0.002458,...,0.011454,0.022840,0.006448,-0.007203,0.029689,0.004942,0.007751,-0.020226,0.028821,0.017492
11750,21751,0.003835,0.015067,0.015428,-0.002030,0.001205,0.012396,0.011026,-0.001491,0.005310,...,0.012303,0.018314,0.012326,-0.012152,0.024617,0.002096,0.001911,-0.008028,0.035093,0.017439
11751,21752,0.000627,0.011407,0.010957,0.000534,-0.000347,0.013499,0.010541,0.001867,0.007447,...,0.012055,0.020741,0.001160,-0.007140,0.025236,0.002026,0.001876,-0.014612,0.021665,0.019592
11752,21753,0.007431,0.021419,0.014143,-0.005623,-0.006399,0.008602,0.006831,-0.001018,0.014972,...,0.019220,0.026282,0.020715,-0.008130,0.025149,0.007695,0.003226,-0.003505,0.029326,0.014235


In [5]:
loading_df.drop(['IC_05','IC_02','IC_24'],axis=1,inplace=True)

,Id,IC_01,IC_07,IC_16,IC_26,IC_06,IC_10,IC_09,IC_18,IC_04,...,IC_08,IC_03,IC_21,IC_28,IC_11,IC_20,IC_30,IC_22,IC_29,IC_14
0,10001,0.006070,0.014466,0.000658,-0.002742,0.005033,0.016720,0.003484,0.001797,0.029223,...,0.018246,0.023711,0.009177,-0.013929,0.030696,0.010496,0.002892,-0.023235,0.022177,0.017192
1,10002,0.009087,0.009291,-0.002076,-0.002227,0.004605,0.012277,0.002946,0.004086,0.027333,...,0.014635,0.022556,0.012004,-0.011814,0.022479,0.005739,0.002880,-0.016609,0.025543,0.014524
2,10003,0.008151,0.014684,-0.005293,-0.002913,0.015042,0.017745,0.003930,-0.008021,0.038530,...,0.019565,0.030616,0.018184,-0.010469,0.029799,0.015435,0.005211,-0.028882,0.031427,0.018164
3,10004,0.004675,0.000957,-0.000429,-0.001222,0.011755,0.013010,0.000193,0.008075,0.027787,...,0.002658,0.022266,0.005956,-0.010595,0.024078,-0.000319,0.005866,-0.015182,0.024476,0.014760
4,10005,-0.000398,0.006878,0.000369,0.000336,0.010679,0.010352,0.003637,0.004180,0.021058,...,0.009703,0.017257,0.005454,-0.008591,0.019416,0.000786,0.002692,-0.019814,0.017105,0.013316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11749,21750,0.005996,0.003873,0.000242,-0.002159,0.020201,0.020931,0.003684,-0.002458,0.033895,...,0.011454,0.022840,0.006448,-0.007203,0.029689,0.004942,0.007751,-0.020226,0.028821,0.017492
11750,21751,0.003835,0.015067,-0.002030,0.001205,0.012396,0.011026,-0.001491,0.005310,0.023853,...,0.012303,0.018314,0.012326,-0.012152,0.024617,0.002096,0.001911,-0.008028,0.035093,0.017439
11751,21752,0.000627,0.011407,0.000534,-0.000347,0.013499,0.010541,0.001867,0.007447,0.020901,...,0.012055,0.020741,0.001160,-0.007140,0.025236,0.002026,0.001876,-0.014612,0.021665,0.019592
11752,21753,0.007431,0.021419,-0.005623,-0.006399,0.008602,0.006831,-0.001018,0.014972,0.042646,...,0.019220,0.026282,0.020715,-0.008130,0.025149,0.007695,0.003226,-0.003505,0.029326,0.014235


In [6]:
fnc_features, loading_features = list(fnc_df.columns[1:]), list(loading_df.columns[1:])
df = fnc_df.merge(loading_df, on="Id")


labels_df = cudf.read_csv("../input/trends-assessment-prediction/train_scores.csv")
labels_df["is_train"] = True

df = df.merge(labels_df, on="Id", how="left")

test_df = df[df["is_train"] != True].copy()
df = df[df["is_train"] == True].copy()

df.shape, test_df.shape

((5877, 1408), (5877, 1408))

In [7]:
#df.drop(['IC_20','IC_05','IC_30','IC_08'],axis=1,inplace=True)


In [8]:
loading_df

,Id,IC_01,IC_07,IC_16,IC_26,IC_06,IC_10,IC_09,IC_18,IC_04,...,IC_08,IC_03,IC_21,IC_28,IC_11,IC_20,IC_30,IC_22,IC_29,IC_14
0,10001,0.006070,0.014466,0.000658,-0.002742,0.005033,0.016720,0.003484,0.001797,0.029223,...,0.018246,0.023711,0.009177,-0.013929,0.030696,0.010496,0.002892,-0.023235,0.022177,0.017192
1,10002,0.009087,0.009291,-0.002076,-0.002227,0.004605,0.012277,0.002946,0.004086,0.027333,...,0.014635,0.022556,0.012004,-0.011814,0.022479,0.005739,0.002880,-0.016609,0.025543,0.014524
2,10003,0.008151,0.014684,-0.005293,-0.002913,0.015042,0.017745,0.003930,-0.008021,0.038530,...,0.019565,0.030616,0.018184,-0.010469,0.029799,0.015435,0.005211,-0.028882,0.031427,0.018164
3,10004,0.004675,0.000957,-0.000429,-0.001222,0.011755,0.013010,0.000193,0.008075,0.027787,...,0.002658,0.022266,0.005956,-0.010595,0.024078,-0.000319,0.005866,-0.015182,0.024476,0.014760
4,10005,-0.000398,0.006878,0.000369,0.000336,0.010679,0.010352,0.003637,0.004180,0.021058,...,0.009703,0.017257,0.005454,-0.008591,0.019416,0.000786,0.002692,-0.019814,0.017105,0.013316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11749,21750,0.005996,0.003873,0.000242,-0.002159,0.020201,0.020931,0.003684,-0.002458,0.033895,...,0.011454,0.022840,0.006448,-0.007203,0.029689,0.004942,0.007751,-0.020226,0.028821,0.017492
11750,21751,0.003835,0.015067,-0.002030,0.001205,0.012396,0.011026,-0.001491,0.005310,0.023853,...,0.012303,0.018314,0.012326,-0.012152,0.024617,0.002096,0.001911,-0.008028,0.035093,0.017439
11751,21752,0.000627,0.011407,0.000534,-0.000347,0.013499,0.010541,0.001867,0.007447,0.020901,...,0.012055,0.020741,0.001160,-0.007140,0.025236,0.002026,0.001876,-0.014612,0.021665,0.019592
11752,21753,0.007431,0.021419,-0.005623,-0.006399,0.008602,0.006831,-0.001018,0.014972,0.042646,...,0.019220,0.026282,0.020715,-0.008130,0.025149,0.007695,0.003226,-0.003505,0.029326,0.014235


In [9]:
#test_df.drop(['IC_20'],axis=1,inplace=True)

In [10]:
#df.drop(['IC_20'],axis=1,inplace=True)

In [11]:
#test_df.drop(['IC_20','IC_05','IC_30','IC_08'],axis=1,inplace=True)

In [12]:
# Giving less importance to FNC features since they are easier to overfit due to high dimensionality.
FNC_SCALE = 1/600
df[fnc_features] *= FNC_SCALE
test_df[fnc_features] *= FNC_SCALE

In [13]:
test_df[fnc_features]

,SCN(53)_vs_SCN(69),SCN(98)_vs_SCN(69),SCN(99)_vs_SCN(69),SCN(45)_vs_SCN(69),ADN(21)_vs_SCN(69),ADN(56)_vs_SCN(69),SMN(3)_vs_SCN(69),SMN(9)_vs_SCN(69),SMN(2)_vs_SCN(69),SMN(11)_vs_SCN(69),...,CBN(13)_vs_DMN(94),CBN(18)_vs_DMN(94),CBN(4)_vs_DMN(94),CBN(7)_vs_DMN(94),CBN(18)_vs_CBN(13),CBN(4)_vs_CBN(13),CBN(7)_vs_CBN(13),CBN(4)_vs_CBN(18),CBN(7)_vs_CBN(18),CBN(7)_vs_CBN(4)
11,0.000790,0.000835,0.001206,0.001041,-0.000654,-0.000251,-0.000448,-0.000586,-0.000325,-0.000902,...,0.000206,0.000430,0.000384,0.000477,0.001110,0.000995,0.001039,0.000484,0.001111,0.000815
12,0.000703,0.000569,0.001180,0.001032,-0.000431,0.000190,0.000084,-0.000173,0.000663,0.000067,...,-0.000101,0.000369,-0.000122,0.000340,0.001025,0.000395,0.000981,0.000153,0.000985,0.000375
13,0.000501,0.000318,0.000748,0.000681,-0.000489,-0.000403,0.000182,-0.000466,-0.000409,-0.000126,...,-0.000634,-0.000013,0.000001,0.000099,0.000713,0.000993,0.000744,0.000411,0.001148,0.000870
14,0.000045,-0.000164,0.000294,0.000517,-0.000351,-0.000372,-0.000356,-0.000130,-0.000178,-0.000268,...,-0.000118,0.000657,0.000397,0.000386,0.000736,0.000691,0.001148,0.000317,0.001224,0.000572
15,0.000401,0.000304,0.000981,0.001116,-0.000031,0.000166,-0.000055,-0.000323,0.000101,0.000041,...,-0.000367,0.000457,-0.000235,0.000432,0.000623,0.000500,0.000599,0.000208,0.001170,0.000357
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11749,0.000592,0.000429,0.000934,0.000276,0.000122,0.000609,0.000178,0.000165,0.000051,0.000105,...,0.000103,0.001035,-0.000186,0.000823,0.000361,0.001027,0.000553,-0.000091,0.000918,0.000456
11750,0.000959,-0.000093,0.000948,0.001086,-0.000757,-0.000329,-0.000183,-0.000347,0.000229,-0.000276,...,-0.000198,0.000969,-0.000240,0.000604,0.000399,0.000495,0.000855,-0.000141,0.000831,0.000503
11751,0.000228,0.000248,0.000985,0.000698,0.000137,0.000143,0.000097,-0.000169,0.000054,0.000084,...,-0.000225,0.000695,-0.000135,0.000780,0.000659,0.000557,0.000339,0.000039,0.001131,0.000144
11752,-0.000325,-0.000154,0.000754,0.000682,-0.000413,-0.000202,-0.000389,-0.000340,0.000019,-0.000020,...,-0.000531,0.000680,-0.000251,0.000742,0.000498,0.000567,0.000169,-0.000030,0.001215,0.000129


In [14]:
%%time

NUM_FOLDS = 7
kf = KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=0)


features = loading_features + fnc_features


overal_score = 0

for target, c, w, ff in [("domain1_var1",12, 0.3, 0.2)]:    
    y_oof = np.zeros(df.shape[0])
    y_test = np.zeros((test_df.shape[0], NUM_FOLDS))

    
    for f, (train_ind, val_ind) in enumerate(kf.split(df, df)):
        train_df, val_df = df.iloc[train_ind], df.iloc[val_ind]
        train_df = train_df[train_df[target].notnull()]

        model_1 = SVR(C=c, cache_size=3000.0)
        model_1.fit(train_df[features].values, train_df[target].values)
        model_2 = Ridge(alpha = 0.0001)
        model_2.fit(train_df[features].values, train_df[target].values)

        val_pred_1 = model_1.predict(val_df[features])
        val_pred_2 = model_2.predict(val_df[features])

        
        test_pred_1 = model_1.predict(test_df[features])
        test_pred_2 = model_2.predict(test_df[features])

            
        val_pred = (1-ff)*val_pred_1+ff*val_pred_2 
        val_pred = cp.asnumpy(val_pred.values.flatten())
        
        test_pred = (1-ff)*test_pred_1 + ff*test_pred_2 
        test_pred = cp.asnumpy(test_pred.values.flatten())
        
        y_oof[val_ind] = val_pred
        y_test[:, f] = test_pred
        
    df["pred_{}".format(target)] = y_oof
    test_df[target] = y_test.mean(axis=1)
    
    score = metric(df[df[target].notnull()][target].values, df[df[target].notnull()]["pred_{}".format(target)].values)
    overal_score += w*score
    print(target, np.round(score, 8))
    print(target, np.round(score, 4))
    print()
    
print("Overal score:", np.round(overal_score, 8))
print("Overal score:", np.round(overal_score, 4))

/opt/conda/envs/rapids/lib/python3.6/site-packages/cuml/utils/input_utils.py:188: UserWarning: Expected column ('F') major order, but got the opposite. Converting data, this will result in additional memory utilization.
  warnings.warn("Expected " + order_to_str(order) + " major order, "
/opt/conda/envs/rapids/lib/python3.6/site-packages/cuml/utils/input_utils.py:188: UserWarning: Expected column ('F') major order, but got the opposite. Converting data, this will result in additional memory utilization.
  warnings.warn("Expected " + order_to_str(order) + " major order, "


domain1_var1 0.15104049
domain1_var1 0.151

Overal score: 0.04531215
Overal score: 0.0453
CPU times: user 29 s, sys: 1.86 s, total: 30.8 s
Wall time: 31 s


In [15]:
sub_df = cudf.melt(test_df[["Id", "age", "domain1_var1", "domain1_var2", "domain2_var1", "domain2_var2"]], id_vars=["Id"], value_name="Predicted")
sub_df["Id"] = sub_df["Id"].astype("str") + "_" +  sub_df["variable"].astype("str")

sub_df = sub_df.drop("variable", axis=1).sort_values("Id")
assert sub_df.shape[0] == test_df.shape[0]*5
sub_df.head(10)

,Id,Predicted
3921,10003_age,null
9798,10003_domain1_var1,50.29063964
15675,10003_domain1_var2,null
21552,10003_domain2_var1,null
27429,10003_domain2_var2,null
3929,10006_age,null
9806,10006_domain1_var1,54.69363224
15683,10006_domain1_var2,null
21560,10006_domain2_var1,null
27437,10006_domain2_var2,null


In [16]:
sub_df.to_csv("submission_rapids_ensemble.csv", index=False)